In [1]:
%pylab inline
plt.style.use("default")
import os, sys
import scipy
import random
import h5py
import pandas as pd
import helpers as h
import spontHelpers as sh
import utils as u
import matplotlib as mpl
from scipy.spatial.distance import pdist, squareform
from glob import glob
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

plt.rcParams['figure.figsize'] = (15, 10) 


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:

def compute_cvPCA(resp0, maxcols=np.inf):
    ss0 = u.shuff_cvPCA(resp0, nshuff=10, maxcols=np.inf)
    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0

In [ ]:
dp0='/camp/home/duuta/working/duuta/ppp0/data/zebf00/TimeSeries.h5'
dp1='/camp/home/duuta/working/duuta/ppp0/data/zebf00/data_full.mat'


In [ ]:
d1 = loadmat(dp1, simplify_cells=True)
dholder = h5py.File(dp0, 'r')
d0 = dholder['CellResp'][:]


In [ ]:
eliminated_rois = d1['data']['IX_inval_anat']

In [ ]:
all_rois = d1['data']['CellXYZ']

In [ ]:
# these are the positions of the rois which have been recorded
usd_rois = [row for j, row in enumerate(all_rois) if j not in list(eliminated_rois)]
used_rois_coor = np.array(usd_rois)

In [ ]:
for key in d1['data'].keys():
    try:
        print(f"{key}: {d1['data'][key].shape}")

    except:
        print(f"{key}: no shape")
        print(f"{key}: {d1['data'][key]}")
        continue


In [ ]:
x, y, z = used_rois[:50000, :].T

In [ ]:
def find_nearest_nbrs(ds, roi_idx, n=10):
    n_idx = np.argsort(ds[roi_idx,])[1:n+1]

    return n_idx

def compute_distance_matrix(x, y):
    ds = squareform(pdist(np.array([x, y]).T, metric='euclidean'))

    return ds


def pick_random_nbrs(roi_idx, len0=100, n=10):

    all_idx= list(range(len0))
    all_idx.remove(roi_idx)
    rsample = random.sample(all_idx, n)

    return rsample



In [ ]:
plt.scatter(x, y)
plt.xlabel('X positions')
plt.xlabel('Y positions')
plt.title('X, Y positions of 50k ROIs')


In [ ]:
ncorr = []
rcorr = []
nrcorr = []
rncorr = []
nn_idxs = []
rn_idxs = []
collect_nc_min_max = []
collect_rn_min_max = []

for ridx in range(50000):
    roi = d0[:, ridx]
    nneigh_idx = find_nearest_nbrs(dS, ridx, n=10)
    nn_roi = d0[:, nneigh_idx]
    ri = pick_random_nbrs(ridx, len0=50000, n=10)
    rn_roi = d0[:, ri]
    nrcorr = []
    rncorr = []
    for j in range(nn_roi.shape[1]):
        nc = np.corrcoef(roi, nn_roi[:, j])[0, 1]
        rc = np.corrcoef(roi, rn_roi[:, j])[0, 1]

        nrcorr.append(nc)
        rncorr.append(rc)
        collect_nc_min_max.append(nc)
        collect_rn_min_max.append(rc)
    nn_idxs.append(nneigh_idx)
    rn_idxs.append(ri)  # need to rerun cell for the rn_idxs
    ncorr.append(nrcorr)
    rcorr.append(rncorr)
    
rnr_arr = np.array(collect_nc_min_max)/ np.array(collect_rn_min_max)
PRN  = round(np.percentile(rnr_arr, 90), 3) 

In [ ]:
refined_rnr = [a for a in rnr_arr if a >0]

In [ ]:
rn90 = [ a for a in rnr_arr if a>0 and a<= PRN]

In [ ]:
ax = plt.axes()
for i in range(10000):
    plt.scatter(x[nn_idxs[i]], y[nn_idxs[i]],marker='.', cmap='rainbow', c=np.array(ncorr[i])/np.array(rcorr[i]), vmin=min(rn90), vmax=max(rn90))

plt.colorbar()
plt.xlabel("X Positions", fontsize=20)
plt.ylabel("Y Positions", fontsize=20)
plt.title("postions of ROIs colored by the ratio of near correlations to random correlations of ROIs", fontsize=20)
ax.set_facecolor('black')

In [ ]:
# checking for correctness
fig0 = plt.figure(layout='constrained')
gs = GridSpec(3, 20, figure=fig0)
# frames for ROIs 
ax0 = fig0.add_subplot(gs[0, :10]) # nn
ax0.plot(d0[:, 0])
ax0.set_title(f'ROI of interest: avg={2}')
ax1 = fig0.add_subplot(gs[0, 10:]) # rn
ax1.plot(d0[:, 0])
ax1.set_title(f'ROI of interest: avg={3}')

nnrois = d0[:,nn_idxs[0]]
# frames of dependent nnROIs
ax2 = fig0.add_subplot(gs[1, 0:2])
ax2.plot(d0[: nnrois[0]])
ax2.set_title(f'ROI of id: avg={3}')

ax3 = fig0.add_subplot(gs[1, 2:4])
ax3.plot(d0[: nnrois[1]])
ax3.set_title()

ax4 = fig0.add_subplot(gs[1, 4:6])
ax4.plot(d0[: nnrois[2]])
ax4.set_title()

ax5 = fig0.add_subplot(gs[1, 6:8])
ax5.plot(d0[: nnrois[3]])
ax5.set_title()

ax6 = fig0.add_subplot(gs[1, 8:10])
ax6.plot(d0[: nnrois[4]])
ax6.set_title()

ax7 = fig0.add_subplot(gs[2, 0:2])
ax7.plot(d0[: nnrois[5]])
ax7.set_title()

ax8 = fig0.add_subplot(gs[2, 2:4])
ax8.plot(d0[: nnrois[6]])
ax8.set_title()

ax9 = fig0.add_subplot(gs[2, 4:6])
ax9.plot(d0[: nnrois[7]])
ax9.set_title()

ax10 = fig0.add_subplot(gs[2, 6:8])
ax10.plot(d0[: nnrois[8]])
ax10.title()

ax11 = fig0.add_subplot(gs[2, 8:10])
ax11.plot(d0[: nnrois[9]])
ax11.set_title()

rnrois = 
# frames dependent rnROIs
ax12 = fig0.add_subplot(gs[1, 10:12])
ax12.plot()
ax12.set_title()

ax13 = fig0.add_subplot(gs[1, 12:14])
ax13.plot()
ax13.set_title('')

ax14 = fig0.add_subplot(gs[1, 14:16])
ax14.plot()
ax14.set_title('')

ax15 = fig0.add_subplot(gs[1, 16:18])
ax15.plot()
ax15.set_title('')

ax16 = fig0.add_subplot(gs[1, 18:20])
ax16.plot()
ax16.set_title('')

ax17 = fig0.add_subplot(gs[2, 10:12])
ax17.plot()
ax17.set_title('')

ax18 = fig0.add_subplot(gs[2, 12:14])
ax18.plot()
ax18.set_title('')

ax19 = fig0.add_subplot(gs[2, 14:16])
ax19.plot()
ax19.set_title('')

ax20 = fig0.add_subplot(gs[2, 16:18])
ax20.plot()
ax20.set_title('')

ax21 = fig0.add_subplot(gs[2, 18:20])
ax21.plot()
ax21.set_title('')


fig0.suptitle('checking correctness')
